In [ ]:
#!huggingface-cli login



In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# model = "meta-llama/Llama-2-7b-chat-hf"
model_id = "meta-llama/Llama-2-7b-hf"

hf_tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    device_map="auto",
)


/home/mazumdera/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.09it/s]


In [20]:
import inspect
inspect.getabsfile(hf_model.__init__)

'/home/mazumdera/.local/lib/python3.10/site-packages/transformers/models/llama/modeling_llama.py'

In [2]:
import torch
test_input_ids = {"input_ids": torch.tensor([[1, 306]])}
test_output = hf_model(**test_input_ids, output_hidden_states=True)


qkv tensor([[[[ 0.1156, -0.4135,  0.7659,  ..., -3.1262,  1.6900, -0.3126],
          [-0.1424,  0.2927, -0.6192,  ...,  0.4953, -0.6919,  0.5496],
          [-0.0293,  0.3192,  0.3883,  ...,  0.3689,  0.3571,  0.3483],
          ...,
          [ 0.6956, -1.5982, -0.4457,  ...,  1.6039,  0.5294,  1.2793],
          [ 0.7316,  1.0250, -0.6231,  ..., -0.4763,  0.5224,  0.4853],
          [-1.3654,  1.3793, -0.2871,  ..., -0.1139,  0.6236,  0.1626]],

         [[-0.0119, -0.9941,  1.5719,  ..., -0.5904,  0.2736, -0.8128],
          [ 0.3898,  0.1954, -1.0632,  ..., -0.3247,  0.0182, -0.2878],
          [-0.5449, -0.7177, -0.8624,  ..., -0.2956, -0.2598, -0.2284],
          ...,
          [ 0.4036, -0.9814, -0.3059,  ...,  1.3979,  0.7752,  1.1634],
          [ 0.6024,  2.1749, -1.3464,  ..., -0.3872,  0.6416,  0.6622],
          [-1.8424, -1.2155, -1.6240,  ..., -0.0669,  0.1928,  0.0739]]]],
       grad_fn=<ViewBackward0>) tensor([[[[-0.4510, -0.0164,  0.0498,  ...,  0.0514, -0.0235, -0.

In [3]:
y= test_output.hidden_states[0]
y.shape

torch.Size([1, 2, 4096])

In [4]:
y1 = hf_model.model.layers[0].input_layernorm(y)

# Self Attention
sa_hidden_states, self_attn_weights, present_key_value = hf_model.model.layers[0].self_attn(
    hidden_states=y1,
)

hidden_states = y + sa_hidden_states

# Fully Connected
residual = hidden_states
hidden_states =  hf_model.model.layers[0].post_attention_layernorm(hidden_states)
inp_mlp = hidden_states
hidden_states = hf_model.model.layers[0].mlp(hidden_states)




qkv tensor([[[[ 0.1156, -0.4135,  0.7659,  ..., -3.1262,  1.6900, -0.3126],
          [-0.1424,  0.2927, -0.6192,  ...,  0.4953, -0.6919,  0.5496],
          [-0.0293,  0.3192,  0.3883,  ...,  0.3689,  0.3571,  0.3483],
          ...,
          [ 0.6956, -1.5982, -0.4457,  ...,  1.6039,  0.5294,  1.2793],
          [ 0.7316,  1.0250, -0.6231,  ..., -0.4763,  0.5224,  0.4853],
          [-1.3654,  1.3793, -0.2871,  ..., -0.1139,  0.6236,  0.1626]],

         [[-0.0119, -0.9941,  1.5719,  ..., -0.5904,  0.2736, -0.8128],
          [ 0.3898,  0.1954, -1.0632,  ..., -0.3247,  0.0182, -0.2878],
          [-0.5449, -0.7177, -0.8624,  ..., -0.2956, -0.2598, -0.2284],
          ...,
          [ 0.4036, -0.9814, -0.3059,  ...,  1.3979,  0.7752,  1.1634],
          [ 0.6024,  2.1749, -1.3464,  ..., -0.3872,  0.6416,  0.6622],
          [-1.8424, -1.2155, -1.6240,  ..., -0.0669,  0.1928,  0.0739]]]],
       grad_fn=<ViewBackward0>) tensor([[[[-0.4510, -0.0164,  0.0498,  ...,  0.0514, -0.0235, -0.

In [12]:
k = hf_model.model.layers[0].self_attn.k_proj(
    y1,
)[:,1,:].view(1, 1, 32, 128).transpose(1, 2)
q = hf_model.model.layers[0].self_attn.q_proj(
    y1,
)[:,1,:].view(1, 1, 32, 128).transpose(1, 2)

q.shape, k.shape

(torch.Size([1, 32, 1, 128]), torch.Size([1, 32, 1, 128]))

In [23]:
q

tensor([[[[-0.0119, -0.9941,  1.5719,  ..., -0.5904,  0.2736, -0.8128]],

         [[ 0.3898,  0.1954, -1.0632,  ..., -0.3247,  0.0182, -0.2878]],

         [[-0.5449, -0.7177, -0.8624,  ..., -0.2956, -0.2598, -0.2284]],

         ...,

         [[ 0.4036, -0.9814, -0.3059,  ...,  1.3979,  0.7752,  1.1634]],

         [[ 0.6024,  2.1749, -1.3464,  ..., -0.3872,  0.6416,  0.6622]],

         [[-1.8424, -1.2155, -1.6240,  ..., -0.0669,  0.1928,  0.0739]]]],
       grad_fn=<TransposeBackward0>)

In [6]:
y[0,1]

tensor([ 0.0097, -0.0013, -0.0182,  ...,  0.0081, -0.0023, -0.0083],
       grad_fn=<SelectBackward0>)

In [8]:
y1[0,1]

tensor([ 0.0266, -0.0017, -0.0033,  ...,  0.0076, -0.0023, -0.0047],
       grad_fn=<SelectBackward0>)

In [9]:
sa_hidden_states[0,1]

tensor([-0.0130, -0.0145, -0.0017,  ...,  0.0167,  0.0091,  0.0181],
       grad_fn=<SelectBackward0>)

In [22]:
self_attn_weights

In [11]:
residual[0,1]

tensor([-0.0033, -0.0158, -0.0199,  ...,  0.0248,  0.0068,  0.0098],
       grad_fn=<SelectBackward0>)

In [13]:
inp_mlp[0,1]

tensor([-0.0065, -0.0324, -0.0388,  ...,  0.0507,  0.0141,  0.0187],
       grad_fn=<SelectBackward0>)

In [15]:
hidden_states[0,1]

tensor([ 0.0189,  0.0090,  0.0092,  ...,  0.0134, -0.0040,  0.0062],
       grad_fn=<SelectBackward0>)

In [22]:
test_output.hidden_states[-1][0,1]

tensor([ 2.1923, -1.0025, -4.2113,  ..., -1.0603, -3.5505, -0.5630],
       grad_fn=<SelectBackward0>)

In [23]:
hf_model.generate(**{"input_ids":torch.tensor([[304]])},do_sample=False, max_new_tokens=4)

qkv tensor([[[[ 1.9113e-01, -3.0842e-01,  9.0489e-01,  ..., -6.4505e-01,
            5.0731e-01, -6.6772e-01],
          [ 3.8363e-01, -2.8364e-02, -1.4495e+00,  ..., -5.1494e-01,
            4.4443e-01, -5.3332e-01],
          [-6.0156e-01, -3.9155e-01, -3.8934e-01,  ..., -4.3788e-01,
           -4.0367e-01, -3.7883e-01],
          ...,
          [ 2.0351e-01, -1.0506e-01,  7.7070e-04,  ...,  9.2187e-01,
            1.4416e-01,  1.9297e-01],
          [ 4.0002e-01,  1.8807e+00, -1.2176e+00,  ..., -3.6590e-01,
            6.1665e-01,  6.3293e-01],
          [-2.0418e+00, -1.3954e+00, -1.7669e+00,  ..., -6.1653e-02,
            2.6229e-02,  3.0898e-01]]]]) tensor([[[[ 0.0313, -0.7540,  0.9622,  ..., -0.4308,  0.3907, -0.4261],
          [ 0.5274,  1.0983,  0.1495,  ...,  0.7061, -0.6655,  0.7036],
          [-0.0570,  0.2481,  0.2665,  ...,  1.3278,  1.3381,  1.2454],
          ...,
          [-1.8050, -0.4950, -1.3538,  ..., -0.7516, -1.6699, -1.3148],
          [-0.5305, -0.5939, -0.6

tensor([[  304,   278, 29871, 29896, 29929]])

In [19]:
import jax
import jax.numpy as jnp

def jax_rope(
    inputs: jax.Array,
    position
) :
    embedding_dims = 128
    if len(inputs.shape) != 4:
        raise ValueError(
            'Input is assumed to be a rank 4 tensor of shape'
            '[batch, sequence, heads, dims].'
        )
    if embedding_dims != inputs.shape[3]:
        raise ValueError(
            'The embedding dims of the rotary position embedding'
            'must match the hidden dimension of the inputs.'
        )
    half_embedding_dim = embedding_dims // 2
    fraction = 2 * jnp.arange(0, half_embedding_dim) / embedding_dims
    timescale = (
        10000 ** fraction
    )
    if position is None:
        seq_length = inputs.shape[1]
        position = jnp.arange(seq_length, dtype=jnp.float32)[jnp.newaxis, :]

    position = position[:, :, jnp.newaxis, jnp.newaxis]
    timescale = timescale[jnp.newaxis, jnp.newaxis, jnp.newaxis, :]
    sinusoid_inp = position / timescale
    sin = jnp.sin(sinusoid_inp)
    cos = jnp.cos(sinusoid_inp)
    reshape_tensor = inputs.astype(jnp.float32).reshape(
        *inputs.shape[:-1], -1, 2
    )
    first_half = reshape_tensor[..., 0]
    second_half = reshape_tensor[..., 1]
    first_part = first_half * cos - second_half * sin
    second_part = second_half * cos + first_half * sin
    
    first_part = first_part.astype(jnp.float32)
    second_part = second_part.astype(jnp.float32)
    x_out = jnp.stack((first_part, second_part), axis=-1).reshape(
        *first_part.shape[:-1], -1
    )
    return x_out, cos, sin

In [28]:
hsin = hf_model.model.layers[0].self_attn.rotary_emb.sin_cached[1]
hcos = hf_model.model.layers[0].self_attn.rotary_emb.cos_cached[1]
fsin = hsin[:64]
fcos = hcos[:64]

In [13]:
def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids, unsqueeze_dim=1):
    """Applies Rotary Position Embedding to the query and key tensors.

    Args:
        q (`torch.Tensor`): The query tensor.
        k (`torch.Tensor`): The key tensor.
        cos (`torch.Tensor`): The cosine part of the rotary embedding.
        sin (`torch.Tensor`): The sine part of the rotary embedding.
        position_ids (`torch.Tensor`):
            The position indices of the tokens corresponding to the query and key tensors. For example, this can be
            used to pass offsetted position ids when working with a KV-cache.
        unsqueeze_dim (`int`, *optional*, defaults to 1):
            The 'unsqueeze_dim' argument specifies the dimension along which to unsqueeze cos[position_ids] and
            sin[position_ids] so that they can be properly broadcasted to the dimensions of q and k. For example, note
            that cos[position_ids] and sin[position_ids] have the shape [batch_size, seq_len, head_dim]. Then, if q and
            k have the shape [batch_size, heads, seq_len, head_dim], then setting unsqueeze_dim=1 makes
            cos[position_ids] and sin[position_ids] broadcastable to the shapes of q and k. Similarly, if q and k have
            the shape [batch_size, seq_len, heads, head_dim], then set unsqueeze_dim=2.
    Returns:
        `tuple(torch.Tensor)` comprising of the query and key tensors rotated using the Rotary Position Embedding.
    """
    cos = cos[position_ids].unsqueeze(unsqueeze_dim)
    sin = sin[position_ids].unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


In [51]:
rq= apply_rotary_pos_emb(q, k, hcos.unsqueeze(0), hsin.unsqueeze(0), torch.tensor([[0]]))[0].view(1,1,32,128)

In [24]:
jpos = jnp.zeros((1,1), dtype=jnp.float32)+1
jq = jnp.array(q.transpose(1,2).detach().numpy())
jq.shape

(1, 1, 32, 128)

In [62]:
inputs = jq
position = jpos
embedding_dims = 128
if len(inputs.shape) != 4:
    raise ValueError(
        'Input is assumed to be a rank 4 tensor of shape'
        '[batch, sequence, heads, dims].'
    )
if embedding_dims != inputs.shape[3]:
    raise ValueError(
        'The embedding dims of the rotary position embedding'
        'must match the hidden dimension of the inputs.'
    )
half_embedding_dim = embedding_dims // 2
fraction = 2 * jnp.arange(0, half_embedding_dim) / embedding_dims
timescale = (
    10000 ** fraction
)
if position is None:
    seq_length = inputs.shape[1]
    position = jnp.arange(seq_length, dtype=jnp.float32)[jnp.newaxis, :]

position = position[:, :, jnp.newaxis, jnp.newaxis]
timescale = timescale[jnp.newaxis, jnp.newaxis, jnp.newaxis, :]
sinusoid_inp = position / timescale
sin = jnp.sin(sinusoid_inp)
cos = jnp.cos(sinusoid_inp)
# reshape_tensor = inputs.astype(jnp.float32).reshape(
#     *inputs.shape[:-1], -1, 2
# )
# first_half = reshape_tensor[..., 0]
# second_half = reshape_tensor[..., 1]
first_half = inputs[...,:half_embedding_dim]
second_half = inputs[...,half_embedding_dim:]
first_part = first_half * cos - second_half * sin
second_part = second_half * cos + first_half * sin

first_part = first_part.astype(jnp.float32)
second_part = second_part.astype(jnp.float32)
x_out = jnp.stack((first_part, second_part), axis=-2).reshape(
    *first_part.shape[:-1], -1
)

In [63]:
x_out

Array([[[[ 1.4565277 , -1.8312544 ,  0.8833891 , ..., -0.5904649 ,
           0.27353516, -0.81288964],
         [ 0.265101  ,  0.09343203, -1.0226979 , ..., -0.32477704,
           0.01808614, -0.2878942 ],
         [-0.88165027, -0.8515903 , -0.07818246, ..., -0.29560813,
          -0.25972414, -0.22832702],
         ...,
         [-0.7708775 , -0.8401944 , -0.88000953, ...,  1.3978256 ,
           0.7750829 ,  1.1633689 ],
         [-1.9961946 ,  2.5168455 , -2.710215  , ..., -0.3873494 ,
           0.6415954 ,  0.66209906],
         [-1.3836857 , -1.193929  , -1.8324177 , ..., -0.06692716,
           0.19277065,  0.07394321]]]], dtype=float32)

In [52]:
rq

tensor([[[[ 1.4565, -1.8313,  0.8834,  ..., -0.5905,  0.2735, -0.8129],
          [ 0.2651,  0.0934, -1.0227,  ..., -0.3248,  0.0181, -0.2879],
          [-0.8817, -0.8516, -0.0782,  ..., -0.2956, -0.2597, -0.2283],
          ...,
          [-0.7709, -0.8402, -0.8800,  ...,  1.3978,  0.7751,  1.1634],
          [-1.9962,  2.5168, -2.7102,  ..., -0.3873,  0.6416,  0.6621],
          [-1.3837, -1.1939, -1.8324,  ..., -0.0669,  0.1928,  0.0739]]]],
       grad_fn=<ViewBackward0>)

In [39]:
import numpy as np
np.testing.assert_allclose(cos[0,0,0,:], fcos.numpy(), atol=1e-4)
np.testing.assert_allclose(sin[0,0,0,:], fsin.numpy(), atol=1e-4)

In [54]:
first_part

Array([[[[ 1.4565277 , -1.8312544 ,  0.8833891 , ..., -0.7160979 ,
          -0.425218  , -0.8340009 ],
         [ 0.265101  ,  0.09343203, -1.0226979 , ..., -0.7126686 ,
          -0.4810583 , -0.7696133 ],
         [-0.88165027, -0.8515903 , -0.07818246, ...,  0.26456898,
           0.23997688,  0.21529286],
         ...,
         [-0.7708775 , -0.8401944 , -0.88000953, ..., -0.29993495,
          -0.6755829 , -0.42354798],
         [-1.9961946 ,  2.5168455 , -2.710215  , ..., -0.6818527 ,
          -0.3995863 , -0.4855073 ],
         [-1.3836857 , -1.193929  , -1.8324177 , ..., -0.47816563,
          -0.12306862,  0.16565482]]]], dtype=float32)

In [43]:
inputs[:,:,:,:64]

Array([[[[-0.01191656, -0.9940782 ,  1.5719095 , ..., -0.7161888 ,
          -0.4251815 , -0.83409476],
         [ 0.3897705 ,  0.1954282 , -1.0632446 , ..., -0.7127186 ,
          -0.4810559 , -0.7696466 ],
         [-0.5448863 , -0.71766275, -0.86243594, ...,  0.26452345,
           0.23994224,  0.21526648],
         ...,
         [ 0.40358144, -0.98136365, -0.3059196 , ..., -0.2997197 ,
          -0.67547953, -0.42341363],
         [ 0.60238856,  2.174935  , -1.3464367 , ..., -0.6819123 ,
          -0.39950073, -0.48543084],
         [-1.8424119 , -1.2154958 , -1.6240358 , ..., -0.4781759 ,
          -0.1230429 ,  0.16566336]]]], dtype=float32)

In [46]:
inputs[:,:,0,:]

Array([[[-0.01191656, -0.9940782 ,  1.5719095 ,  0.6162712 ,
          1.2939439 , -1.8479637 ,  1.3882785 , -0.10092734,
         -1.5812262 ,  1.2490873 , -1.2330507 ,  1.3749135 ,
         -1.5649699 , -0.54764307,  1.055608  ,  0.8287907 ,
          1.3092983 , -0.13034879,  0.11129674, -0.7954103 ,
          0.67359257,  0.72003734, -1.1377401 , -0.73731506,
         -0.6121722 , -0.7499683 ,  0.53513277,  0.51578546,
         -0.6382929 , -0.1064329 ,  0.5784683 ,  0.36507058,
          1.285033  ,  0.7751149 , -1.2202055 ,  1.3085903 ,
          1.1058295 , -0.9379916 , -0.19292709,  0.60728073,
         -0.8949725 ,  1.2462053 ,  0.41698945,  0.1773171 ,
         -0.03962983, -0.46832195,  1.3028119 ,  1.2415118 ,
          1.4311945 , -1.3384933 , -1.297977  ,  1.157537  ,
          1.0662632 , -0.6498189 ,  0.69216585,  0.93620104,
          0.91583055,  0.29048637, -0.32514098,  0.5350121 ,
         -0.46832862, -0.7161888 , -0.4251815 , -0.83409476,
         -1.7385819 ,  1

In [45]:
reshape_tensor[:,:,0,:,:]

Array([[[[-0.01191656, -0.9940782 ],
         [ 1.5719095 ,  0.6162712 ],
         [ 1.2939439 , -1.8479637 ],
         [ 1.3882785 , -0.10092734],
         [-1.5812262 ,  1.2490873 ],
         [-1.2330507 ,  1.3749135 ],
         [-1.5649699 , -0.54764307],
         [ 1.055608  ,  0.8287907 ],
         [ 1.3092983 , -0.13034879],
         [ 0.11129674, -0.7954103 ],
         [ 0.67359257,  0.72003734],
         [-1.1377401 , -0.73731506],
         [-0.6121722 , -0.7499683 ],
         [ 0.53513277,  0.51578546],
         [-0.6382929 , -0.1064329 ],
         [ 0.5784683 ,  0.36507058],
         [ 1.285033  ,  0.7751149 ],
         [-1.2202055 ,  1.3085903 ],
         [ 1.1058295 , -0.9379916 ],
         [-0.19292709,  0.60728073],
         [-0.8949725 ,  1.2462053 ],
         [ 0.41698945,  0.1773171 ],
         [-0.03962983, -0.46832195],
         [ 1.3028119 ,  1.2415118 ],
         [ 1.4311945 , -1.3384933 ],
         [-1.297977  ,  1.157537  ],
         [ 1.0662632 , -0.6498189 ],
 

In [47]:
reshape_tensor[:,:,0,:,0]

Array([[[-0.01191656,  1.5719095 ,  1.2939439 ,  1.3882785 ,
         -1.5812262 , -1.2330507 , -1.5649699 ,  1.055608  ,
          1.3092983 ,  0.11129674,  0.67359257, -1.1377401 ,
         -0.6121722 ,  0.53513277, -0.6382929 ,  0.5784683 ,
          1.285033  , -1.2202055 ,  1.1058295 , -0.19292709,
         -0.8949725 ,  0.41698945, -0.03962983,  1.3028119 ,
          1.4311945 , -1.297977  ,  1.0662632 ,  0.69216585,
          0.91583055, -0.32514098, -0.46832862, -0.4251815 ,
         -1.7385819 ,  0.39155698,  0.57888097, -1.3691111 ,
         -1.7069377 , -0.15189448,  0.05268119,  1.0003893 ,
          0.43506047,  0.84984046, -0.8323429 ,  0.83382165,
          0.59543097, -0.7101423 ,  0.68925995,  1.112483  ,
         -0.37695333,  0.36179817, -1.0613778 , -1.4701483 ,
          0.5386509 , -0.7990422 ,  1.1357825 ,  1.0592446 ,
         -1.4061098 , -0.90385103,  0.7719959 , -0.45916808,
         -0.5233414 ,  0.77508986,  0.8089928 ,  0.27359185]]],      dtype=float32)

In [28]:
hf_model.model.embed_tokens.weight[0]

tensor([ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
        -6.5565e-06,  8.9407e-07], grad_fn=<SelectBackward0>)

In [29]:
hf_model.lm_head.weight[0]

tensor([-0.0039,  0.0032, -0.0071,  ...,  0.0053, -0.0082,  0.0070],
       grad_fn=<SelectBackward0>)